In [105]:
import pandas as pd
from collections import defaultdict
import copy

# INPUT : network structure
net = pd.read_table('/data/Gitcode/230206_structure1.tsv', sep = '\t')
grouped = net.loc[:,['source_genesymbol','target_genesymbol','consensus_inhibition']].groupby(['target_genesymbol','consensus_inhibition']).aggregate(list)
targetdict = defaultdict(list)

# Make regulator logics
for target in grouped.index:
    if target[1] == 1 :
            targetdict[target[0]].append('!' + '('+ ' | '.join(grouped.loc[target,:].values[0]) + ')')
    else:
        targetdict[target[0]].append('('+ ' & '.join(grouped.loc[target,:].values[0]) + ')')

In [133]:
# Modify logics

user_defined = """
EGF, EGF
KRAS, MET | EGFR
Proliferation, RPS6KB1 | (MYC & !CDKN1A) 
Apoptosis, (!BCL2 | !MAPK1 & !MAPK3) & (FOXO3 | !MAPK1 & TP53)

EGFR, EGF | ABL1 & (!CDK1 | !PTEN)
MET, ABL1
MAPK1, !PPP1CA | MAP2K1 | MAP2K2 | AKT3
MAPK3, !PPP1CA & MAPK1 | MAP2K1 | MAP2K2
CRK, (!EGFR & ABL1) | (IRS1 & PIK3R1)
AKT1, IRS1 & PIN1 | MTOR & (PIK3CA | PIK3R1 | PRKDC)
AKT3, (PTEN & MTOR) & !(PPP1CA)
BAD, !(AKT1 | AKT2 | AKT3) | !(MAP2K1 | MAPK1 | MAPK3) | !(MAPK8 | PAK1 | RPS6KB1) & CDK1 & PPP1CA
BCL2, BCL2L1 & !BAD & !(BCL2L11 | CDK1 | NR4A1 | TP53) & !(MAPK14 | MAPK8) | (BCL2L1 & MAPK1)
BRAF, !AKT1 | !(MAPK1 & MAPK3 & RHEB) | KRAS & PAK1
"""

userdict = {x.split(',')[0]: x.split(',')[1:] for x in user_defined.split('\n') if x != ''}

targetdict2 = copy.deepcopy(targetdict)
for k in userdict.keys():
    targetdict2[k] = userdict[k]

In [134]:
source = ['&'.join(v) for v in targetdict2.values()]
out0 = []
for k in targetdict2.keys():
    if k not in ''.join(source):
        out0.append(k)
out0.remove('Proliferation')
out0.remove('Apoptosis')
print(out0)

['ABL2', 'CTNND1', 'GATA1', 'PIK3CB', 'PRKAG2']


In [135]:
bnet = ''
for k,v in targetdict2.items():
    if (k not in out0) & (len(v) > 0):
        bnet += (k + ', ' + ' & '.join(v) + '\n')

In [136]:
print(bnet)

ABL1, (PRKDC & PAK1 & CDK1)
AKT1, (PIK3R1 & MTOR & PRKDC & PIK3CA & IRS1)
AKT2, (PRKDC & MTOR & PIK3CA) & !(PPP1CA | AKT1)
AKT3, (PTEN & MTOR) & !(PPP1CA)
BAD, (PPP1CA & CDK1) & !(AKT1 | MAPK8 | PAK1 | AKT1 | AKT3 | AKT2 | MAPK3 | MAPK1 | MAP2K1 | RPS6KB1)
BCL2, (MAPK1 & BCL2L1) & !(MAPK8 | TP53 | BCL2L11 | BAD | CDK1 | MAPK3 | MAPK14 | NR4A1 | AKT3 | AKT1)
BCL2L1, (AKT1) & !(BCL2L11 | BAD | MAPK8 | TP53)
BCL2L11, (MAPK8 & E2F1) & !(FOXO3 | MAPK3 | MAPK1 | AKT1 | AKT3 | AKT2)
BRAF, (KRAS & PAK1) & !(MAPK1 | MAPK3 | AKT3 | AKT1 | AKT2 | RHEB)
CDK1, !(GSK3B | ERBB2 | CDKN1A)
CDK2, (AKT1 & MAPK1)
CDK5, (ABL1)
CDKN1A, (DAPK3 & SP1 & TP53 & MAPK14 & MAPK8 & SMAD2 & SMAD3) & !(AKT2 | AKT1 | MYC | CDK2 | AKT1 | AKT3 | MAPK1 | MAPK3 | GSK3B | MDM2 | FOXO3)
CEBPA, !(MAPK1 | MAPK3 | MYC)
CHUK, (AKT2 & AKT3 & AKT1)
CREB1, (AKT1 & SMAD2 & JUN & AKT2 & AKT3 & MAPK1 & MAPK3) & !(GSK3B | MAPK14 | NR3C1)
CRK,  (!EGFR & ABL1) | (IRS1 & PIK3R1)
CTNNB1, (EP300 & PAK1 & AKT1 & CREB1 & AKT3 & AKT2 & ERBB2 